In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import cvzone
from detection import DETECT
from mss import mss
from deep_sort_realtime.deepsort_tracker import DeepSort
import datetime

# Detect a Video

In [2]:
DETECT(model_path = 'weights/v10_120_s.pt', 
       is_screen_capture=True,
       show_output = True, 
       #video_path = "/Users/oscarwan/bballDetection/videos/videos_clipped/shot.mp4",
       save_output = False,
       #output_path="v9.mp4",
       verbose = True
       )


0: 384x640 3 basketballs, 2 basketball-hoopss, 482.5ms
Speed: 20.7ms preprocess, 482.5ms inference, 45.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 basketballs, 2 basketball-hoopss, 31.7ms
Speed: 4.5ms preprocess, 31.7ms inference, 6.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 29.9ms
Speed: 4.0ms preprocess, 29.9ms inference, 45.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 basketballs, 29.9ms
Speed: 3.0ms preprocess, 29.9ms inference, 29.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 1 basketball-hoops, 27.3ms
Speed: 2.6ms preprocess, 27.3ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 basketball, 30.0ms
Speed: 3.2ms preprocess, 30.0ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.9ms
Speed: 3.0ms preprocess, 28.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no 

KeyboardInterrupt: 

In [2]:
model_path = 'weights/v10_120_s.pt'
is_screen_capture = False
video_path = "/Users/oscarwan/bballDetection/videos/videos_clipped/shot.mp4"
show_output = True,
save_output = False,
output_path = "output.mp4",
verbose = True

In [3]:
CONFIDENCE_THRESHOLD = 0.8
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)

# initialize the video capture object
video_cap = cv2.VideoCapture(video_path)


# load the pre-trained YOLOv8n model
model = YOLO(model_path)
print("Model loaded successfully!")
tracker = DeepSort(max_age=10)


while True:
    start = datetime.datetime.now()

    ret, frame = video_cap.read()

    if not ret:
        break

    # run the YOLO model on the frame
    detections = model(frame, device = "mps")[0]

    # initialize the list of bounding boxes and confidences
    results = []

    ######################################
    # DETECTION
    ######################################

    # loop over the detections
    for data in detections.boxes.data.tolist():
        # extract the confidence (i.e., probability) associated with the prediction
        confidence = data[4]

        # filter out weak detections by ensuring the 
        # confidence is greater than the minimum confidence
        if float(confidence) < CONFIDENCE_THRESHOLD:
            continue

        # if the confidence is greater than the minimum confidence,
        # get the bounding box and the class id
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        class_id = int(data[5])
        # add the bounding box (x, y, w, h), confidence and class id to the results list
        results.append([[xmin, ymin, xmax - xmin, ymax - ymin], confidence, class_id])

    ######################################
    # TRACKING
    ######################################

    # update the tracker with the new detections
    tracks = tracker.update_tracks(results, frame=frame)
    # loop over the tracks
    for track in tracks:
        # if the track is not confirmed, ignore it
        if not track.is_confirmed():
            continue

        # get the track id and the bounding box
        track_id = track.track_id
        ltrb = track.to_ltrb()

        xmin, ymin, xmax, ymax = int(ltrb[0]), int(
            ltrb[1]), int(ltrb[2]), int(ltrb[3])
        # draw the bounding box and the track id
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), GREEN, 2)
        cv2.rectangle(frame, (xmin, ymin - 20), (xmin + 20, ymin), GREEN, -1)
        cv2.putText(frame, str(track_id), (xmin + 5, ymin - 8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, WHITE, 2)

    # end time to compute the fps
    end = datetime.datetime.now()
    # show the time it took to process 1 frame
    print(f"Time to process 1 frame: {(end - start).total_seconds() * 1000:.0f} milliseconds")
    # calculate the frame per second and draw it on the frame
    fps = f"FPS: {1 / (end - start).total_seconds():.2f}"
    cv2.putText(frame, fps, (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)

    # show the frame to our screen
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) == ord("q"):
        break

video_cap.release()
cv2.destroyAllWindows()

Model loaded successfully!


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.

0: 384x640 2 basketballs, 3 basketball-hoopss, 508.9ms
Speed: 18.5ms preprocess, 508.9ms inference, 38.2ms postprocess per image at shape (1, 3, 384, 640)


Time to process 1 frame: 3949 milliseconds



0: 384x640 2 basketballs, 3 basketball-hoopss, 32.5ms
Speed: 2.4ms preprocess, 32.5ms inference, 6.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 basketballs, 3 basketball-hoopss, 28.7ms
Speed: 2.3ms preprocess, 28.7ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)


Time to process 1 frame: 134 milliseconds



0: 384x640 2 basketballs, 3 basketball-hoopss, 29.9ms
Speed: 2.1ms preprocess, 29.9ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)


: 

: 

In [4]:
!pip install deep-sort-realtime

  Using cached deep_sort_realtime-1.3.2-py3-none-any.whl (8.4 MB)


In [1]:
import json

a
